## STEP03. 청킹 UDF 만들기

검색 모델은 소규모 텍스트 청크에서 가장 잘 작동하므로 긴 문서를 Cortex Search에 제공하면 성능이 저하됩니다. 그런 다음 Python UDF를 생성하여 텍스트를 청크한다.

청킹 이란 : https://wikidocs.net/265566

In [ ]:
CREATE OR REPLACE FUNCTION cortex_search_tutorial_db.public.books_chunk(
    description string, title string, authors string, category string, publisher string
)
    returns table (chunk string, title string, authors string, category string, publisher string)
    language python
    runtime_version = '3.9'
    handler = 'text_chunker'
    packages = ('snowflake-snowpark-python','langchain')
    as
$$
from langchain.text_splitter import RecursiveCharacterTextSplitter
import copy
from typing import Optional

class text_chunker:

    def process(self, description: Optional[str], title: str, authors: str, category: str, publisher: str):
        if description == None:
            description = "" # handle null values

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size = 2000,
            chunk_overlap  = 300,
            length_function = len
        )
        chunks = text_splitter.split_text(description)
        for chunk in chunks:
            yield (title + "\n" + authors + "\n" + chunk, title, authors, category, publisher) # always chunk with title
$$;

## STEP04.청크 테이블 만들기

기록에서 추출한 텍스트 청크를 저장할 테이블을 생성합니다. 청크에 제목과 화자를 포함하여 컨텍스트를 제공

In [ ]:
CREATE TABLE cortex_search_tutorial_db.public.book_description_chunks AS (
    SELECT
        books.*,
        t.CHUNK as CHUNK
    FROM cortex_search_tutorial_db.public.books_dataset_raw books,
        TABLE(cortex_search_tutorial_db.public.books_chunk(books.description, books.title, books.authors, books.category, books.publisher)) t
);

테이블 내용을 확인합니다.

In [ ]:
SELECT chunk, * FROM book_description_chunks LIMIT 10;

## STEP05.Cortex Search Service 만들기

**book_description_chunks** 의 청크를 검색할 수 있도록 테이블에 Cortex Search Service를 생성

In [ ]:
CREATE CORTEX SEARCH SERVICE cortex_search_tutorial_db.public.books_dataset_service
    ON CHUNK
    WAREHOUSE = cortex_search_tutorial_wh
    TARGET_LAG = '1 hour'
    AS (
        SELECT *
        FROM cortex_search_tutorial_db.public.book_description_chunks
    );